# Основы построения рекомендательных систем

## Домашнее задание №2
Подготовить валидацию с использованием фреймворка RecTools. Работаем с датасетом Kion.
- [Документация RecTools](https://rectools.readthedocs.io/en/latest/)
- [Экзамплы](https://github.com/MobileTeleSystems/RecTools/tree/main/examples). Для данного домашнего задания хватит первых двух ноутбуков

**Важно!** Разработанный функционал будет использоваться вами на протяжении всего курса, это ваш основной инструмент для валидации всех моделей, которые вы будете изучать.

**Задача:** разработать две функции (по желанию можно реализовать классами) - одна для расчёта метрик на основе кросс-валидации и одна для визуального анализа рекомендаций.

### Расчёт метрик (5 баллов)
1. Функция будет принимать на вход:
  1. Словарь с инициализированными моделями
  2. Словарь с инициализированными метриками
  3. Инициализированный Splitter для кросс-валидации
  4. Количество рекомендаций для генерации (K)

2. Реализация обучения и валидации:
  1. Создаем RecTools Dataset через метод construct на train взаимодействиях для каждого фолда
  2. Обучаем модель (не забываем сделать deepcopy), рекоменуем K айтемов для каждого юзера, считаем метрики на test
  3. Дополнительно логируем время обучения
  4. Сохраняем метрики в отчёт

3. Результат оборачиваем в pandas DataFrame и усредняем по фолдам

### Визуальный анализ (5 баллов)
1. Функция будет принимать на вход:
  1. Инициализированную (и обученную!) модель
  2. Датасет
  3. Список отобранных user_id для просмотра
  4. item_data - данные об айтемах, которые важно отразить для визуального анализа (напр. название)

2. Реализация:
  1. Генерим рекомендации для отобранных user_id
  2. Получаем из датасета истории просмотров юзеров
  3. Отображаем в любом удобном виде, который позволит смотреть на историю просмотра юзера + на его рекомендации от модели

Для тестирования функций используем:
1. Модели: rectools.models.RandomModel(random_state=32), rectools.models.PopularModel() с параметрами по умолчанию
2. Метрики: 2 ранжирующие, 2 классификационные, 2 beyond-accuracy. Считаем по порогам 1, 5, 10. MAP обязательно
3. Сплиттер: rectools.model_selection.TimeRangeSplitter, 3 фолда для кросс-валидации по неделе, исключение холодных юзеров и айтемов и просмотренных айтемов
4. Визуализация рекомендаций и историй просмотров для юзеров [666262, 672861, 955527]. Для айтемов обязательно отражаем названия, жанры и количество просмотров в датасете (как для айтемов из истории взаимодействий каждого юзера, так и для айтемов из его рекомендаций)

Весь результат в одном чистом, воспроизводимом jupyter ноутбуке

Напоминаем ссылку на датасет: \
url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

In [1]:
! pip install rectools

In [2]:
import requests
from tqdm.auto import tqdm
import zipfile as zf

import pandas as pd
import numpy as np

from rectools import Columns
from rectools.dataset import Dataset, Interactions
from rectools.metrics import MAP, MeanInvUserFreq, NDCG, Precision, Recall, Serendipity, calc_metrics
from rectools.models import PopularModel, RandomModel
from rectools.model_selection import TimeRangeSplitter

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [3]:
# Функция валидирования моделей методом Leave-Time-out
def validate(models: dict, metrics: dict, splitter: TimeRangeSplitter, dataset: Dataset, K_RECOS: int):
  results = []

  # создаем итератор фолдов
  fold_iterator = splitter.split(dataset.interactions, collect_fold_stats=True)

  # обучение по фолдам
  for train_ids, test_ids, fold_info in tqdm((fold_iterator), total=splitter.n_splits):
      print(f"\n==================== Fold {fold_info['i_split']} ====================")
      print(fold_info)

      # тренировочная часть
      df_train = dataset.interactions.df.iloc[train_ids]
      train_dataset = Dataset.construct(df_train)

      # тестовая часть
      df_test = dataset.interactions.df.iloc[test_ids][Columns.UserItem]
      test_users = np.unique(df_test[Columns.User])

      # каталог
      catalog = df_train[Columns.Item].unique()

      # обучение моделей
      for model_name, model in models.items():
          # обучение и получение рекомендаций
          model.fit(train_dataset)
          recos = model.recommend(
              users=test_users,
              dataset=train_dataset,
              k=K_RECOS,
              filter_viewed=True,
          )
          # подсчет метрик
          metric_values = calc_metrics(
              metrics,
              reco=recos,
              interactions=df_test,
              prev_interactions=df_train,
              catalog=catalog,
          )
          # сохранение результатов
          res = {"fold": fold_info["i_split"], "model": model_name}
          res.update(metric_values)
          results.append(res)

  # форматирование в DataFrame
  pivot_results = pd.DataFrame(results).drop(columns="fold").groupby(["model"], sort=False).agg("mean")
  # сохранение отчета
  pivot_results.to_csv('./report.csv')

  return pivot_results

In [4]:
# Функция визуального анализа
def visualize(model: None, dataset: Dataset, sample_users: list, item_data: list=None, K_RECOS: int=10):
  model_name = model[0]
  model = model[1]

  if item_data:
    users_data = dataset.interactions.df[dataset.interactions.df[Columns.User].isin(sample_users)].sort_values("user_id").reset_index(drop=True)[item_data]
  else:
    users_data = dataset.interactions.df[dataset.interactions.df[Columns.User].isin(sample_users)].sort_values("user_id").reset_index(drop=True)

  recos = model.recommend(
      users=sample_users,
      dataset=dataset,
      k=K_RECOS,
      filter_viewed=True
      )

  print("=" * 20 + f" Модель {model_name} " + "=" * 20)
  print(f"Users {sample_users}", end='\n'*2)
  print("*" * 10 + f" История просмотров юзеров " + "*" * 10)
  print(users_data, end='\n'*2)
  print("*" * 10 + f" Рекомендации для юзеров " + "*" * 10)
  print(recos, end='\n'*4)

## Чтение данных

In [5]:
# Установка архива
url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'
req = requests.get(url, stream=True)

with open('kion.zip', 'wb') as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

kion dataset download:   0%|          | 0.00/78.8M [00:00<?, ?iB/s]

In [6]:
# Разархивирование данных
files = zf.ZipFile('kion.zip','r')
files.extractall()
files.close()

In [7]:
# Чтение в DataFrame
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')

interactions = pd.read_csv('data_original/interactions.csv', parse_dates=["last_watch_dt"])
interactions.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        'total_dur': Columns.Weight
    },
    inplace=True)

In [8]:
users.head(5)

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


In [9]:
items.head(3)

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."


In [10]:
interactions.head(5)

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


## Модели

In [11]:
dataset = Dataset.construct(interactions_df=interactions)

models = {'random_model': RandomModel(random_state=32),
          'pop_model': PopularModel()}

## Валидация и расчет метрик

In [12]:
metrics = {
    "prec@1": Precision(k=1),
    "prec@5": Precision(k=5),
    "prec@10": Precision(k=10),
    "recall@1": Recall(k=1),
    "recall@5": Recall(k=5),
    "recall@10": Recall(k=10),
    "MAP@1": MAP(k=1),
    "MAP@5": MAP(k=5),
    "MAP@10": MAP(k=10),
    "NDCG@1": MAP(k=1),
    "NDCG@5": MAP(k=5),
    "NDCG@10": MAP(k=10),
    "MeanInvUserFreq@1": MeanInvUserFreq(k=1),
    "MeanInvUserFreq@5": MeanInvUserFreq(k=5),
    "MeanInvUserFreq@10": MeanInvUserFreq(k=10),
    "Serendipity@1": Serendipity(k=1),
    "Serendipity@5": Serendipity(k=5),
    "Serendipity@10": Serendipity(k=10),
}


In [13]:
n_splits = 3
cv = TimeRangeSplitter(test_size="7D",
                        n_splits=n_splits,
                        filter_cold_users=True,
                        filter_cold_items=True,
                        filter_already_seen=True)

In [14]:
%%time
K_RECOS = 10

pivot_results = validate(models=models,
                   metrics=metrics,
                   splitter=cv,
                   dataset=dataset,
                   K_RECOS=K_RECOS)

  0%|          | 0/3 [00:00<?, ?it/s]


==================== Fold 0 ====================
{'i_split': 0, 'start': Timestamp('2021-08-02 00:00:00', freq='7D'), 'end': Timestamp('2021-08-09 00:00:00', freq='7D'), 'train': 4266013, 'train_users': 797423, 'train_items': 15237, 'test': 263681, 'test_users': 98184, 'test_items': 6602}

==================== Fold 1 ====================
{'i_split': 1, 'start': Timestamp('2021-08-09 00:00:00', freq='7D'), 'end': Timestamp('2021-08-16 00:00:00', freq='7D'), 'train': 4649162, 'train_users': 850489, 'train_items': 15415, 'test': 279422, 'test_users': 103511, 'test_items': 6698}

==================== Fold 2 ====================
{'i_split': 2, 'start': Timestamp('2021-08-16 00:00:00', freq='7D'), 'end': Timestamp('2021-08-23 00:00:00', freq='7D'), 'train': 5051815, 'train_users': 906071, 'train_items': 15577, 'test': 298878, 'test_users': 110076, 'test_items': 6679}
CPU times: user 1min 48s, sys: 2.87 s, total: 1min 51s
Wall time: 1min 51s


In [15]:
pivot_results.style \
    .highlight_min(subset=pivot_results.columns, color='coral', axis=0) \
    .highlight_max(subset=pivot_results.columns , color='green', axis=0)

,prec@1,recall@1,prec@5,recall@5,prec@10,recall@10,MAP@1,MAP@5,MAP@10,NDCG@1,NDCG@5,NDCG@10,MeanInvUserFreq@1,MeanInvUserFreq@5,MeanInvUserFreq@10,Serendipity@1,Serendipity@5,Serendipity@10
model,,,,,,,,,,,,,,,,,,
random_model,0.000157,0.000058,0.000171,0.000322,0.000173,0.000685,0.000058,0.000144,0.000191,0.000058,0.000144,0.000191,15.607427,15.611645,15.610776,0.000003,0.000005,0.000006
pop_model,0.076432,0.042720,0.052402,0.137413,0.033903,0.173492,0.042720,0.078295,0.084109,0.042720,0.078295,0.084109,2.377055,3.066979,3.713390,0.000002,0.000003,0.000002


## Визуальный анализ

In [16]:
sample_users = [666262, 672861, 955527]

In [17]:
# обучаем модели на всем датасете
for model_name, model in models.items():
    model.fit(dataset)

In [18]:
for model in models.items():
  visualize(model, dataset, sample_users, K_RECOS=K_RECOS)

==================== Модель random_model ====================
Users [666262, 672861, 955527]

********** История просмотров юзеров **********
   user_id  item_id    weight   datetime
0   666262       93    2435.0 2021-07-21
1   672861       25  110883.0 2021-07-26
2   672861       32   12662.0 2021-08-01
3   955527       21   19820.0 2021-07-20

********** Рекомендации для юзеров **********
    user_id  item_id  score  rank
0    666262    10101     10     1
1    666262      619      9     2
2    666262    12618      8     3
3    666262     5967      7     4
4    666262     4041      6     5
5    666262     5701      5     6
6    666262     9738      4     7
7    666262    15247      3     8
8    666262    10004      2     9
9    666262     2816      1    10
10   672861     9457     10     1
11   672861    15730      9     2
12   672861      473      8     3
13   672861    12736      7     4
14   672861     3927      6     5
15   672861     3300      5     6
16   672861     5334      4 